In [1]:
import logging
logging.basicConfig(format='%(asctime)s %(message)s')

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)




import requests
import json
import sys
from api_insee import ApiInsee
from api_insee.criteria import Field

import secret 


import pandas as pd
logger.info("END OF CONF")

    


2020-11-21 17:15:46,619 END OF CONF


In [2]:
logger.info("Start")


api = ApiInsee(
    key = secret.sirene['key'],
    secret = secret.sirene['secret']
)

logging.debug("Api Created")

#data = api.siret(q=(Field('codeCommuneEtablissement', 92046) | Field('unitePurgeeUniteLegale', True))).get()
#print (data)
tot=0
data = api.siret(q=(Field('codeCommuneEtablissement', "38*") , Field('etatAdministratifUniteLegale', "A"),  Field('categorieJuridiqueUniteLegale', "5*"),
                   Field('activitePrincipaleUniteLegale', "16*")))

d=data.get()["header"]["total"]
logging.info("Nb entreprises : " + str(d))

2020-11-21 17:17:04,518 Start
2020-11-21 17:17:05,345 Api Created
2020-11-21 17:17:06,461 Nb entreprises : 274


In [4]:
pd.json_normalize(data.get()["etablissements"])

,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,...,adresse2Etablissement.libelleVoie2Etablissement,adresse2Etablissement.codePostal2Etablissement,adresse2Etablissement.libelleCommune2Etablissement,adresse2Etablissement.libelleCommuneEtranger2Etablissement,adresse2Etablissement.distributionSpeciale2Etablissement,adresse2Etablissement.codeCommune2Etablissement,adresse2Etablissement.codeCedex2Etablissement,adresse2Etablissement.libelleCedex2Etablissement,adresse2Etablissement.codePaysEtranger2Etablissement,adresse2Etablissement.libellePaysEtranger2Etablissement
0,352533921,00198,35253392100198,O,1996-12-31,12,2018,None,2020-08-25T10:12:44,False,...,None,None,None,None,None,None,None,None,None,None
1,332106319,00046,33210631900046,O,2007-04-02,02,2018,None,2020-08-25T10:35:24,False,...,None,None,None,None,None,None,None,None,None,None
2,402695845,00014,40269584500014,O,1995-10-23,NN,None,None,2008-12-06T04:16:14,True,...,None,None,None,None,None,None,None,None,None,None
3,799852603,00015,79985260300015,O,2014-01-01,00,2017,None,2019-06-24T13:53:35,False,...,None,None,None,None,None,None,None,None,None,None
4,799792874,00023,79979287400023,O,2016-09-01,02,2018,3299ZG,2020-08-25T11:09:09,True,...,None,None,None,None,None,None,None,None,None,None
5,403857188,00011,40385718800011,O,1996-01-02,00,2018,1610AQ,2020-08-25T10:35:51,True,...,None,None,None,None,None,None,None,None,None,None
6,423972249,00019,42397224900019,O,1999-09-01,NN,None,None,2014-07-08T02:01:41,False,...,None,None,None,None,None,None,None,None,None,None
7,453214389,00018,45321438900018,O,2004-05-01,NN,None,None,2017-05-04T02:50:45,False,...,None,None,None,None,None,None,None,None,None,None
8,453867137,00011,45386713700011,O,2004-05-14,NN,None,None,2019-11-14T14:00:36,False,...,None,None,None,None,None,None,None,None,None,None
9,441614617,00039,44161461700039,O,2009-10-01,NN,None,None,2016-10-12T03:50:14,True,...,None,None,None,None,None,None,None,None,None,None


In [5]:
logger.setLevel(logging.INFO)
FIELDS = ["siren", "trancheEffectifsEtablissement", "uniteLegale.denominationUniteLegale", "uniteLegale.categorieEntreprise", "uniteLegale.activitePrincipaleUniteLegale"]
tot=0
final=345803
etablissements=[]
logging.info("START PARSING ")
for (page_index, page_result) in enumerate(data.pages(nombre=10000)):
    logging.info("Next page "+str(page_index))
    et=page_result['etablissements']
    tot=tot+len(et)

    logging.info("... +"+str(len(et))+" = "+str(tot))
    if len(et) >0 :
        etablissements.append(pd.json_normalize(et))

logging.info(len(etablissements))
for e in etablissements:
    logging.debug("START ETABLISSEMENT ......")
    logging.debug(len(e))
    logging.debug(e["uniteLegale.categorieEntreprise"].value_counts())
    #logging.info(e)
    #e[FIELDS]    
    logging.debug("END ETABLISSEMENT ......")

tous=pd.concat(etablissements)
tous[FIELDS]



2020-11-21 17:17:51,654 START PARSING 
2020-11-21 17:17:54,764 Next page 0
2020-11-21 17:17:54,766 ... +274 = 274
2020-11-21 17:17:55,342 Next page 1
2020-11-21 17:17:55,345 ... +0 = 274
2020-11-21 17:17:55,347 1


,siren,trancheEffectifsEtablissement,uniteLegale.denominationUniteLegale,uniteLegale.categorieEntreprise,uniteLegale.activitePrincipaleUniteLegale
0,056501729,None,SCIERIE BARTHELEMY FRERES,PME,16.10A
1,056501729,03,SCIERIE BARTHELEMY FRERES,PME,16.10A
2,057503336,00,ETABLISSEMENTS LANFREY,PME,16.10A
3,058503657,12,SCIERIE EYMARD SA,PME,16.10A
4,059500660,NN,SOCIETE FORESTIERE DU DAUPHINE,PME,16.10A
...,...,...,...,...,...
269,851382077,None,MOTTA CUCINE 64,None,16.23Z
270,853411924,None,SCIERIE BOTTAREL,None,16.10A
271,889129219,None,EMBALLAGES DU GUIERS,None,16.24Z
272,950402743,02,SCIERIE DE CHARTREUSE,PME,16.10A


In [6]:
tous["uniteLegale.categorieEntreprise"].value_counts()


tous.groupby(['uniteLegale.activitePrincipaleUniteLegale'])[['siren']].count()
#grouped_multiple.columns = ['age_mean', 'age_min', 'age_max']
#grouped_multiple = grouped_multiple.reset_index()


,siren
uniteLegale.activitePrincipaleUniteLegale,
16.10A,65
16.10B,33
16.21Z,2
16.23Z,78
16.24Z,57
16.29Z,39


In [7]:
tous['activite'] = tous['uniteLegale.activitePrincipaleUniteLegale'].str[:3]

In [8]:
tous.pivot_table(values="siren", index="activite",columns="uniteLegale.categorieEntreprise", aggfunc="count")

uniteLegale.categorieEntreprise,ETI,GE,PME
activite,,,
16.,8,5,218


In [ ]:
logging.info(data.get()["etablissements"])

In [9]:
etablissements=data.get()['etablissements']
df = pd.DataFrame.from_dict(etablissements)

mld= pd.json_normalize(etablissements)
mld[FIELDS]
#logging.info(mld)

KeyError: "None of [Index(['siren', 'trancheEffectifsEtablissement',\n       'uniteLegale.denominationUniteLegale',\n       'uniteLegale.categorieEntreprise',\n       'uniteLegale.activitePrincipaleUniteLegale'],\n      dtype='object')] are in the [columns]"

In [11]:
mld["categorie"].value_counts()


KeyError: 'categorie'

In [ ]:
mld["uniteLegale.activitePrincipaleUniteLegale"].value_counts()


In [ ]:
champs = [
    'siret',
    'denominationUniteLegale',
    'nomUsageUniteLegale',
    'prenom1UniteLegale',
]

data = api.siret('39860733300059').get()
logging.info(data)

In [ ]:
import requests
result = requests.get("https://geo.api.gouv.fr/departements/38/communes")
communes=result.json()
lc=[]
for c in communes:
    lc.append(c["code"])
listeCommunes=','.join(lc)
#print(response.json())